In [58]:
import tensorflow as tf
import pandas as pd
sample_size = 60
training_runs = 150

train = pd.read_csv("train.csv", usecols =['bone_length','rotting_flesh','hair_length','has_soul','type'])
test = pd.read_csv("test.csv", usecols =['id','bone_length','rotting_flesh','hair_length','has_soul'])

x = tf.placeholder(tf.float32, [None, 4])
W = tf.Variable(tf.zeros([4, 3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 3])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(training_runs):
    rand_rows = train.sample(n=sample_size)
    batch_xs = rand_rows.ix[:, :4]
    batch_ys = pd.get_dummies(rand_rows['type']) #.map(['Ghost','Ghoul','Goblin'])   
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

test_inputs = test.ix[:,1:]
test_output = sess.run([y], feed_dict={x:test_inputs})

#print(y)
#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [95]:
test_result = test.ix[:,:1]
test_result['type']=test_result['id']

for i in range(0,len(test_output[0])):
    if test_output[0][i][0] == max(test_output[0][i]):
        test_result['type'][i] = 'Ghost'
    elif test_output[0][i][1] == max(test_output[0][i]):
        test_result['type'][i] = 'Ghoul'
    else:
        test_result['type'][i] = 'Goblin'

/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [96]:
test_result.to_csv("subm.csv", index=False)